Answer 3.

a) They don't output a probability. They only output a classification based on a threshold which gets met based on learned weights that can't be interpreted, so the perceptron can only give us yes or no, and they don't tell us anything intuitive about the system under inspection.

b) You can set the activation function to a logistic function. 

Answer 4.

Logistic activations were attractive because they are continuous and differentiable over the entire real line. They 

Answer 5.

There's the ReLU activation function, the step (heaviside function), the logistic function, and the tanh function. 

Yes. I can draw them.

Answer 6.


1.) With n instances, the shape of the input matrix is n $\times$ 10.    
2.) W is $10 \times 50$ in shape. b is $1 \times 50$    
3.) W is $50 \times 3$ b is $1 \times 3$    
4.) The output matrix has shape $n \times 3$    
5.) $Y = ReLU(ReLU(XW_0+ B_0)W_h+b_h)$


Answer 7.

It takes one neuron in the ouptut layer to classify email as ham or spam. You need 10 neurons to classify the mnist data

To predict housing prices, there are two ways to do this. One is to "categorize" shousing prices into increasingly expensive categories, and this will take $n$ neurons for $n$ levels of expense. For a regression prediction, you only need one neuron.

Answer 8.

Backpropagation is a training algorithm. It efficiently computes gradients w.r.t. all weights automatically. Batches of inputs are sent through the Neural network in multiple passes called epochs. The inputs are passed through all the layers of the network, and individual outputs are saved. Output error is measured, and each output connection's contribution to the error is computed using the chain rule. The error contriburtions from the connections in the layer below are then calculated using chain rule, and a gradient descent step is then used to tweak the weights in the network.  

Answer 9. 

Hyperparameters in a basic MLP:

1.) number of input neurons: can be changed by adding features or by compressing the data    
2.) number of hidden layers    
3.) neurons per (in each) hidden layer    
4.) Hidden layer activation function    
5.) Output actiation    
6.) Loss function    

6.) loss functions include sparse_categorical_crossenropy, mean_squared_error    
    __[Keras loss function documentation](https://keras.io/api/losses/)__

### Decision to Scale

The data was unscaled in a preliminary attempt. I'm leaving the choice here because a <tt> uint8 </tt> takes up $\frac{1}{4}$ the space of a <tt> float</tt>, so the spatial savings might be worth a try without scaling the data.    

Since scaling can lead to improvements, and preliminary models only marginally missed the mark, we're going to retry the same tests without scaling. You can retry the tests without scaling by setting to_scale_data to <tt>False</tt>

In [1]:
to_scale_data = True

In [2]:
from tensorflow import keras

In [3]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

(X_train, y_train), (X_test, y_test) = keras.datasets.mnist.load_data()
if to_scale_data:
    X_train = X_train/255.0
    X_test = X_test/255.0

In [4]:
def basic_mlp_mnist(n_input_neurons=784, n_hidden_layers=2, l_neurons=[30,30], h_activ="relu", opt="adam", met="accuracy"):
    in_l = keras.layers.Flatten(input_shape=[28,28])
    hidden_ls = [ keras.layers.Dense(l_neurons[i], activation=h_activ) for i in range(n_hidden_layers) ]
    out_l = keras.layers.Dense(10, activation="softmax")
    model = keras.models.Sequential([in_l] + hidden_ls + [out_l] )
    model.compile(loss = "sparse_categorical_crossentropy", optimizer=opt, metrics=[met])
    return model

In [5]:
# function that trains @param mod with a tuple data=(X_train, y_train)
# val: validation set
def train_mlp(data, mod, eps=25, val = None, cbs=None):
    if val:
        if cbs:
            cb_in = cbs if isinstance(cbs, list) else [cbs]
            hist = mod.fit(data[0], data[1], epochs=eps, validation_data=val, callbacks=cb_in)
        else:
            hist = mod.fit(data[0], data[1], epochs=eps, validation_data=val)        
    else:
        hist = mod.fit(data[0], data[1], epochs=eps)
    return hist

In [6]:
model0 = basic_mlp_mnist()
history0 = train_mlp((X_train, y_train), model0)

Epoch 1/25
1875/1875 [==============================] - 2s 1ms/step - loss: 0.3497 - accuracy: 0.8997
Epoch 2/25
1875/1875 [==============================] - 2s 1ms/step - loss: 0.1750 - accuracy: 0.9481
Epoch 3/25
1875/1875 [==============================] - 2s 1ms/step - loss: 0.1377 - accuracy: 0.9598
Epoch 4/25
1875/1875 [==============================] - 2s 1ms/step - loss: 0.1171 - accuracy: 0.9651
Epoch 5/25
1875/1875 [==============================] - 2s 1ms/step - loss: 0.1023 - accuracy: 0.9688
Epoch 6/25
1875/1875 [==============================] - 2s 1ms/step - loss: 0.0923 - accuracy: 0.9722
Epoch 7/25
1875/1875 [==============================] - 2s 1ms/step - loss: 0.0838 - accuracy: 0.9734
Epoch 8/25
1875/1875 [==============================] - 2s 1ms/step - loss: 0.0758 - accuracy: 0.9769
Epoch 9/25
1875/1875 [==============================] - 2s 1ms/step - loss: 0.0695 - accuracy: 0.9784
Epoch 10/25
1875/1875 [==============================] - 2s 1ms/step - loss: 0.064

In [7]:
model0.evaluate(X_test, y_test)

313/313 [==============================] - 0s 899us/step - loss: 0.1546 - accuracy: 0.9654


[0.15464189648628235, 0.965399980545044]

We can see that on our first attempt, with 2 layers, we got 94.5% accuracy. oof. Let's try adding more layers

In [8]:
model1 = basic_mlp_mnist( n_hidden_layers=5, l_neurons=[30,30,30,30,30])
history1 = train_mlp((X_train, y_train), model1)
model1.evaluate(X_test, y_test)

Epoch 1/25
1875/1875 [==============================] - 3s 2ms/step - loss: 0.3767 - accuracy: 0.8833
Epoch 2/25
1875/1875 [==============================] - 3s 2ms/step - loss: 0.1722 - accuracy: 0.9485
Epoch 3/25
1875/1875 [==============================] - 3s 2ms/step - loss: 0.1370 - accuracy: 0.9588
Epoch 4/25
1875/1875 [==============================] - 5s 3ms/step - loss: 0.1170 - accuracy: 0.9642
Epoch 5/25
1875/1875 [==============================] - 5s 3ms/step - loss: 0.1026 - accuracy: 0.9686
Epoch 6/25
1875/1875 [==============================] - 6s 3ms/step - loss: 0.0953 - accuracy: 0.9705
Epoch 7/25
1875/1875 [==============================] - 5s 3ms/step - loss: 0.0855 - accuracy: 0.9733
Epoch 8/25
1875/1875 [==============================] - 5s 3ms/step - loss: 0.0807 - accuracy: 0.9747
Epoch 9/25
1875/1875 [==============================] - 5s 3ms/step - loss: 0.0726 - accuracy: 0.9769
Epoch 10/25
1875/1875 [==============================] - 5s 3ms/step - loss: 0.069

[0.15432047843933105, 0.9635999798774719]

In [9]:
model1.evaluate(X_test, y_test)

313/313 [==============================] - 1s 2ms/step - loss: 0.1543 - accuracy: 0.9636


[0.15432047843933105, 0.9635999798774719]

Tantalizingly close. We'll try 10 more epochs

In [10]:
history_1_1 = train_mlp((X_train, y_train), model1, eps=10)

Epoch 1/10
1875/1875 [==============================] - 4s 2ms/step - loss: 0.0326 - accuracy: 0.9892
Epoch 2/10
1875/1875 [==============================] - 4s 2ms/step - loss: 0.0343 - accuracy: 0.9888
Epoch 3/10
1875/1875 [==============================] - 5s 2ms/step - loss: 0.0321 - accuracy: 0.9903
Epoch 4/10
1875/1875 [==============================] - 5s 2ms/step - loss: 0.0321 - accuracy: 0.9898
Epoch 5/10
1875/1875 [==============================] - 5s 2ms/step - loss: 0.0291 - accuracy: 0.9902
Epoch 6/10
1875/1875 [==============================] - 4s 2ms/step - loss: 0.0294 - accuracy: 0.9902
Epoch 7/10
1875/1875 [==============================] - 4s 2ms/step - loss: 0.0304 - accuracy: 0.9900
Epoch 8/10
1875/1875 [==============================] - 4s 2ms/step - loss: 0.0275 - accuracy: 0.9909
Epoch 9/10
1875/1875 [==============================] - 4s 2ms/step - loss: 0.0285 - accuracy: 0.9908
Epoch 10/10
1875/1875 [==============================] - 4s 2ms/step - loss: 0.024

In [11]:
model1.evaluate(X_test, y_test)

313/313 [==============================] - 1s 2ms/step - loss: 0.1522 - accuracy: 0.9659


[0.15219201147556305, 0.9659000039100647]

We got some imporvement by simply including more layers. We'll try some best practices here and split our training set into training and validation. We'll use the <tt> train_test_split </tt> and then pass it to our training function.

In [12]:
from sklearn.model_selection import train_test_split
#bp for best practices
#We'll be using default parameters here. We can 
X_train_bp, X_valid, y_train_bp, y_valid = train_test_split(X_train, y_train)

In [13]:
model2 = basic_mlp_mnist( n_hidden_layers=5, l_neurons=[30,30,30,30,30])
history2 = train_mlp((X_train_bp, y_train_bp), model2, val=(X_valid, y_valid))
model2.evaluate(X_test, y_test)

Epoch 1/25
1407/1407 [==============================] - 4s 3ms/step - loss: 0.4338 - accuracy: 0.8638 - val_loss: 0.2550 - val_accuracy: 0.9218
Epoch 2/25
1407/1407 [==============================] - 5s 3ms/step - loss: 0.1986 - accuracy: 0.9403 - val_loss: 0.1915 - val_accuracy: 0.9409
Epoch 3/25
1407/1407 [==============================] - 5s 4ms/step - loss: 0.1557 - accuracy: 0.9527 - val_loss: 0.1879 - val_accuracy: 0.9419
Epoch 4/25
1407/1407 [==============================] - 5s 4ms/step - loss: 0.1309 - accuracy: 0.9598 - val_loss: 0.1550 - val_accuracy: 0.9526
Epoch 5/25
1407/1407 [==============================] - 5s 3ms/step - loss: 0.1155 - accuracy: 0.9646 - val_loss: 0.1557 - val_accuracy: 0.9521
Epoch 6/25
1407/1407 [==============================] - 5s 4ms/step - loss: 0.0995 - accuracy: 0.9699 - val_loss: 0.1388 - val_accuracy: 0.9589
Epoch 7/25
1407/1407 [==============================] - 5s 4ms/step - loss: 0.0932 - accuracy: 0.9714 - val_loss: 0.1430 - val_accuracy:

[0.16135148704051971, 0.9674999713897705]

There seems to be more information in the tests than we've gotten so far. We'll try using a deeper neural network

In [14]:
model3 = basic_mlp_mnist( n_hidden_layers=9, l_neurons=[30,30,30,30,30,30,30,30,30])
history3 = train_mlp((X_train_bp, y_train_bp), model3, val=(X_valid, y_valid))
model3.evaluate(X_test, y_test)

Epoch 1/25
1407/1407 [==============================] - 6s 4ms/step - loss: 0.5625 - accuracy: 0.8198 - val_loss: 0.2770 - val_accuracy: 0.9175
Epoch 2/25
1407/1407 [==============================] - 6s 4ms/step - loss: 0.2477 - accuracy: 0.9278 - val_loss: 0.2373 - val_accuracy: 0.9322
Epoch 3/25
1407/1407 [==============================] - 7s 5ms/step - loss: 0.1852 - accuracy: 0.9450 - val_loss: 0.2420 - val_accuracy: 0.9274
Epoch 4/25
1407/1407 [==============================] - 8s 6ms/step - loss: 0.1530 - accuracy: 0.9546 - val_loss: 0.1893 - val_accuracy: 0.9443
Epoch 5/25
1407/1407 [==============================] - 9s 7ms/step - loss: 0.1355 - accuracy: 0.9608 - val_loss: 0.1559 - val_accuracy: 0.9543
Epoch 6/25
1407/1407 [==============================] - 8s 6ms/step - loss: 0.1228 - accuracy: 0.9637 - val_loss: 0.1624 - val_accuracy: 0.9543
Epoch 7/25
1407/1407 [==============================] - 7s 5ms/step - loss: 0.1105 - accuracy: 0.9677 - val_loss: 0.1504 - val_accuracy:

[0.17131701111793518, 0.961899995803833]

Both the training and validation errors seem to be decreasing as the number of epochs passed increases. We'll try to improve this model by training the network for more epochs. We'll make sure to use some kind of early stopping to make sure we don't go too far.

In [15]:
model3.save("m3_mnist.h5")

In [16]:
model_3_cb = keras.callbacks.ModelCheckpoint("m3_mnist.h5", save_best_only=True)
m3_hist = train_mlp((X_train_bp, y_train_bp), model3, val=(X_valid, y_valid), cbs = [model_3_cb] )

Epoch 1/25
1407/1407 [==============================] - 6s 4ms/step - loss: 0.0453 - accuracy: 0.9869 - val_loss: 0.1871 - val_accuracy: 0.9597
Epoch 2/25
1407/1407 [==============================] - 6s 5ms/step - loss: 0.0446 - accuracy: 0.9871 - val_loss: 0.1647 - val_accuracy: 0.9613
Epoch 3/25
1407/1407 [==============================] - 7s 5ms/step - loss: 0.0461 - accuracy: 0.9869 - val_loss: 0.1802 - val_accuracy: 0.9589
Epoch 4/25
1407/1407 [==============================] - 6s 5ms/step - loss: 0.0436 - accuracy: 0.9869 - val_loss: 0.1702 - val_accuracy: 0.9631
Epoch 5/25
1407/1407 [==============================] - 6s 4ms/step - loss: 0.0412 - accuracy: 0.9878 - val_loss: 0.1875 - val_accuracy: 0.9627
Epoch 6/25
1407/1407 [==============================] - 6s 4ms/step - loss: 0.0403 - accuracy: 0.9879 - val_loss: 0.1936 - val_accuracy: 0.9545
Epoch 7/25
1407/1407 [==============================] - 6s 5ms/step - loss: 0.0408 - accuracy: 0.9878 - val_loss: 0.1789 - val_accuracy:

We might not see any improvement just from the initial epochs or from the inclusion of the validation set, so we'll try to use wider layers. This increases the danger of overfitting, but it's worth a shot anyways.

In [17]:
model4 = basic_mlp_mnist( n_hidden_layers=5, l_neurons=[100,100,100,100,100])
history1 = train_mlp((X_train, y_train), model4)
model4.evaluate(X_test, y_test)

Epoch 1/25
1875/1875 [==============================] - 6s 3ms/step - loss: 0.2598 - accuracy: 0.9202
Epoch 2/25
1875/1875 [==============================] - 6s 3ms/step - loss: 0.1170 - accuracy: 0.9651
Epoch 3/25
1875/1875 [==============================] - 6s 3ms/step - loss: 0.0902 - accuracy: 0.9728
Epoch 4/25
1875/1875 [==============================] - 6s 3ms/step - loss: 0.0710 - accuracy: 0.9783
Epoch 5/25
1875/1875 [==============================] - 6s 3ms/step - loss: 0.0601 - accuracy: 0.9817
Epoch 6/25
1875/1875 [==============================] - 6s 3ms/step - loss: 0.0527 - accuracy: 0.9840
Epoch 7/25
1875/1875 [==============================] - 6s 3ms/step - loss: 0.0452 - accuracy: 0.9866
Epoch 8/25
1875/1875 [==============================] - 6s 3ms/step - loss: 0.0405 - accuracy: 0.9877
Epoch 9/25
1875/1875 [==============================] - 6s 3ms/step - loss: 0.0378 - accuracy: 0.9886
Epoch 10/25
1875/1875 [==============================] - 6s 3ms/step - loss: 0.031

[0.11329898983240128, 0.977400004863739]

In [18]:
model4.evaluate(X_test, y_test)

313/313 [==============================] - 1s 2ms/step - loss: 0.1133 - accuracy: 0.9774


[0.11329898983240128, 0.977400004863739]

This model looks promising. We can use a validation set to try to better train a model with this architecture.

In [ ]:
model4_bp = basic_mlp_mnist( n_hidden_layers=5, l_neurons=[300,100,100,100,100])
history1 = train_mlp((X_train_bp, y_train_bp), model4_bp,  val=(X_valid, y_valid))
model4_bp.evaluate(X_test, y_test)

Epoch 1/25
1407/1407 [==============================] - 6s 5ms/step - loss: 0.2594 - accuracy: 0.9207 - val_loss: 0.1248 - val_accuracy: 0.9644
Epoch 2/25
1407/1407 [==============================] - 6s 4ms/step - loss: 0.1139 - accuracy: 0.9660 - val_loss: 0.1279 - val_accuracy: 0.9623
Epoch 3/25
1407/1407 [==============================] - 6s 4ms/step - loss: 0.0811 - accuracy: 0.9754 - val_loss: 0.1050 - val_accuracy: 0.9704
Epoch 4/25
1407/1407 [==============================] - 6s 4ms/step - loss: 0.0641 - accuracy: 0.9810 - val_loss: 0.1151 - val_accuracy: 0.9708
Epoch 5/25
1407/1407 [==============================] - 6s 4ms/step - loss: 0.0539 - accuracy: 0.9842 - val_loss: 0.0994 - val_accuracy: 0.9729
Epoch 6/25
1237/1407 [=========================>....] - ETA: 0s - loss: 0.0446 - accuracy: 0.9861

In [ ]:
model4_bp.evaluate(X_test, y_test)

We seem close. We're going to try to use a smaller learning rate to finish up the neural net.

In [ ]:
q_opt = keras.optimizers.Adam(learning_rate = 0.0005)
model4_bp = basic_mlp_mnist( n_hidden_layers=5, l_neurons=[300,300,100,100,100], opt=q_opt)
history1 = train_mlp((X_train_bp, y_train_bp), model4_bp,  val=(X_valid, y_valid))
model4_bp.evaluate(X_test, y_test)


In [ ]:
model4_bp_cb = keras.callbacks.ModelCheckpoint("model4_bp.h5", save_best_only=True)
history4_mp = train_mlp((X_train_bp, y_train_bp), model4_bp,  val=(X_valid, y_valid), cbs=model4_bp_cb )
model4_bp = keras.models.load_model("model4_bp.h5")
model4_bp.evaluate(X_test, y_test)

In [ ]:
model4_bp_cb = keras.callbacks.ModelCheckpoint("model4_bp.h5", save_best_only=True)
history4_mp = train_mlp((X_train_bp, y_train_bp), model4_bp, eps=50, val=(X_valid, y_valid), cbs=model4_bp_cb )
model4_bp = keras.models.load_model("model4_bp.h5")
model4_bp.evaluate(X_test, y_test)

In [ ]:
model4_bp.evaluate(X_test, y_test)

In [ ]:

q_opt = keras.optimizers.Adam(learning_rate = 0.00005)


model4_bp = basic_mlp_mnist( n_hidden_layers=5, l_neurons=[300,300,100,100,100], opt=q_opt, h_activ="tanh")
history1 = train_mlp((X_train_bp, y_train_bp), model4_bp, val=(X_valid, y_valid), eps=50)
model4_bp.evaluate(X_test, y_test)


In [ ]:
model5 = basic_mlp_mnist( n_hidden_layers=3, l_neurons=[100,100,100])
history5 = train_mlp((X_train, y_train), model5)
model5.evaluate(X_test, y_test)

In [ ]:
model5.evaluate(X_test, y_test)

In [ ]:
model6 = basic_mlp_mnist( n_hidden_layers=2, l_neurons=[300,300])
history6 = train_mlp((X_train, y_train), model6)
model6.evaluate(X_test, y_test)

In [ ]:
model6 = basic_mlp_mnist( n_hidden_layers=2, l_neurons=[300,50])
history6 = train_mlp((X_train, y_train), model6)
model6.evaluate(X_test, y_test)